In [5]:
# 네이버 블로그 크롤링 - 제주맛집

In [6]:
from selenium import webdriver
import time
import pandas as pd 
from urllib.parse import quote

In [7]:
url = f'https://search.naver.com/search.naver?where=view&sm=tab_jum&query={quote("제주맛집")}'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
time.sleep(2)

In [8]:
# Blog 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.keys import Keys

body = driver.find_element_by_tag_name('body')
for _ in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [9]:
lis = driver.find_elements_by_css_selector('li.bx._svp_item')
len(lis)

110

In [10]:
li = lis[0]
title = li.find_element_by_css_selector('a.api_txt_lines.total_tit._cross_trigger').text.strip()
title

'만족뿐인 제주 맛집 공유'

In [11]:
date = li.find_element_by_css_selector('.sub_time.sub_txt').text.strip()
date

'2021.11.19.'

In [12]:
lis[1].find_element_by_css_selector('.sub_time.sub_txt').text.strip()

'1시간 전'

In [13]:
href = li.find_element_by_css_selector('a.api_txt_lines.total_tit._cross_trigger').get_attribute('href')
href

'https://blog.naver.com/tok4/222573156818'

In [14]:
from datetime import datetime, timedelta

today = datetime.now() #// .today도 가능
today

datetime.datetime(2021, 12, 8, 17, 18, 12, 858267)

In [15]:
today.year, today.month, today.day

(2021, 12, 8)

In [16]:
#어제 
date = today - timedelta(1)
f'{date.year}.{date.month:02d}.{date.day:02d}.'

'2021.12.07.'

In [17]:
date = today - timedelta(6)
f'{date.year}.{date.month:02d}.{date.day:02d}.'

'2021.12.02.'

In [18]:
#13시간전
date = today - timedelta(hours=13) #시간은 타임 델타에서 hour를 붙여준다
f'{date.year}.{date.month:02d}.{date.day:02d}.'

'2021.12.08.'

In [19]:
date = today - timedelta(minutes=45) #분은 타임 델타에서 minutes를 붙여준다
f'{date.year}.{date.month:02d}.{date.day:02d}.'

'2021.12.08.'

- 날짜처리

In [20]:
from datetime import datetime, timedelta

def convert_date(s):
    if len(s) == 11:
        return s[:-1]

    today = datetime.now()
    if '어제' in s:
        date = today - timedelta(1)
    elif '일' in s:
        date = today - timedelta(int(s[:-3]))
    elif '시간' in s:
        date = today - timedelta(hours=int(s[:-4]))
    elif '분' in s:
        date = today - timedelta(minutes=int(s[:-3]))

    return f'{date.year}.{date.month:02d}.{date.day:02d}'


In [21]:
convert_date('45분전')

'2021.12.08'

- 반복문으로 모든 데이터 가져오기

In [22]:
lines = []
for li in lis:
    title = li.find_element_by_css_selector('a.api_txt_lines.total_tit._cross_trigger').text.strip()
    date = li.find_element_by_css_selector('.sub_time.sub_txt').text.strip()
    desc = li.find_element_by_css_selector('.api_txt_lines.dsc_txt').text.strip()
    href = li.find_element_by_css_selector('a.api_txt_lines.total_tit._cross_trigger').get_attribute('href')
    lines.append([title,convert_date(date),desc,href])

df = pd.DataFrame(lines, columns=['제목','날짜','내용','href'])
df[['제목','날짜','내용']].head()

제목          날짜  \
0   만족뿐인 제주 맛집 공유  2021.11.19   
1       확실한 제주 맛집  2021.12.08   
2  북적였던 제주 맛집 리스트  2021.11.13   
3   감격했던 제주 맛집 소개  2021.11.15   
4    또 생각났던 제주 맛집  2021.12.02   

                                                  내용  
0  지난 주말 친구들과 제주여행을 떠났다가 이름알려진 제주 맛집에 갔다왔어요. 전복뚝배...  
1  싶었던 제주 맛집으로 나들이를 가게 되었고 최상급 퀄리티인 흑돼지는 부위별로 골고루...  
2  무엇보다 비린내 한번 느껴지지 않아 배가 부른 상태였음에도 쉽사리 남길 수가 없었던...  
3  얼마 전 친구들과 제주로 여행갔다가 현지인께 추천받은 제주 맛집에 다녀왔어요. 갓 ...  
4  제가 시킨 제주 비빔국수는 갖은 야채들이 그릇에 알록달록 예쁘게 가득 채워져 있었는...

In [23]:
df.to_csv('네이버-제주맛집.csv', index=False)

In [24]:
driver.close()